In [ ]:
from langchain_core.documents import Document
from database import Database


OpenTelemetry Tracing Details
|  Phoenix Project: personAIble
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.

INSTRUMENTED


In [5]:
db = Database()
google_id = '117296122312982831056'

In [31]:
def get_all_followup_qa_summaries(google_id, database) -> list[dict[str, str]]:
    # get all summaries from the QA table where google_id matches
    qa_summaries = database.db_client.table('QA').select('summary').eq('google_id', google_id).execute()
    return qa_summaries.data

def followupLoader(google_id, database):
    # get all rows from the QA table where google_id matches
    qa_summaries = get_all_followup_qa_summaries(google_id, database)
    for summary in qa_summaries:
        yield Document(page_content=summary['summary'], metadata={"source": "followup"})

def get_all_onboarding_data(google_id, database):
    onboarding_cols = database.db_client.table('onboarding').select('*').eq('google_id', google_id).execute()
    return onboarding_cols.data[0]

def onboardingLoader(google_id, database):
    skipCols = set(['google_id', 'created_at', 'id'])
    onboarding_cols = get_all_onboarding_data(google_id, database)
    for question, answer in onboarding_cols.items():
        if question not in skipCols:
                yield Document(page_content=f"{question} : {answer}", metadata={"source": "onboarding"})

In [ ]:
def getUserDocuments(google_id):
    # load all columns from the users row in the onboarding table
    onboardingLoader = onboardingLoader(google_id, db)
    followupLoader = followupLoader(google_id, db)
    
    # for each QA for google_id, load the summaries

    # Combine documents
    documents = []
    for loader in [onboardingLoader, followupLoader]:
        documents.extend(loader.load())
    
    # Create and initialize model
    model = PersonAIble(k = len(documents))
    model.load_data(documents)
    
    return model